In [67]:
# Block 2
# RAG with title -> article-specific search

In [68]:
api_key = 'up_JQDg7exNaCa9WsUszuo1v3dmWc2LZ'

In [69]:
from langchain_upstage import UpstageEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

llm = ChatUpstage(api_key=api_key, model='solar-1-mini-chat', temperature=0) # added temperature for random seed

embeddings = UpstageEmbeddings(
    api_key=api_key,
    model="embedding-query"
)

In [70]:
query = "사회체육학과 소속 학생이 취득할 수 있는 학위의 종류는 무엇인가요? (A) 공학사 (B) 문학사 (C) 국문학사 (D) 체육학사 (E) 보건학사 (F) 가정학사 (G) 이학사"

In [71]:
def search_title_article_similarity(embedded_title, embedded_article, query):
    
    query_vec = embeddings.embed_query(query)
    
    with open(embedded_title, 'r', encoding='utf-8') as infile:
        data = json.load(infile)
        
    similarities = []
    
    for idx, entry in data.items():
        title_vec = entry.get("title")
        
        query_vec = np.array(query_vec)
        title_vec = np.array(title_vec[0])
        
        # print(query_vec.shape, title_vec.shape)
        
        similarity = cosine_similarity([query_vec], [title_vec])[0][0]
        similarities.append((idx, similarity))
        
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_3_indices = [sim[0] for sim in similarities[:3]]
    
    top1_title_idx, top2_title, top3_title = top_3_indices[0], top_3_indices[1], top_3_indices[2]
    
    with open(embedded_article, 'r', encoding='utf-8') as infile:
        data = json.load(infile)
        
    article_similarities = []
    
    print("Content idx: ", top1_title_idx)
    
    for idx in top_3_indices:
        content = data.get(str(idx), {}).get("content", [])
        
        for article in content:
            print("article no.", article["article_no."])
            article_no = article["article_no."]
            article_vec = np.array(article["embedded_content"])
            
            print(article_vec[0].shape, query_vec.shape)
            
            similarity = cosine_similarity([query_vec], [article_vec[0]])[0][0]
            article_similarities.append((article_no, similarity))
        
        article_similarities.sort(key=lambda x: x[1], reverse=True)
        top_3_article_indices = [sim[0] for sim in article_similarities[:3]]
        
        break # use only top1 title
        
    return top1_title_idx, top_3_article_indices

    """
    "1": {
        "content": [
            {
                "article_no.": 0,
                "embedded_content": [
                    [
    """

In [72]:
embedded_title = "../docs/embedded_title.json"
embedded_article = "../docs/embedded_doc2.json"
# embedded_doc = "../docs/embedded_doc1.json"

title_idx, top3_article_idx= search_title_article_similarity(embedded_title, embedded_article, query)

Content idx:  17
article no. 0
(4096,) (4096,)


In [73]:
def get_article_content_text(title_idx, article_top3_list, doc_2):
    with open(doc_2, 'r', encoding = 'utf-8') as infile:
        data = json.load(infile)
        
    article_contents = data.get(str(title_idx), {}).get("content", [])
    
    top3_article_contents = []
    top3_article_contents.append(article_contents[article_top3_list[0]])
    
    return top3_article_contents

In [74]:
get_article_content_text(title_idx, top3_article_idx, '../docs/doc_2.json')

["부칙(2006. 6. 28  개정)\n ① (시행일)  이 학칙은 2007년 3월 1일부터 시행한다. 다만, 제8조 제4항 제18호는 2005년 8월 13일부터, 제50조 [별표 2] 중 바이오인포매틱스 연계전공은 2006년 9월 1일부터 시행한다. \n② (경과조치)  제50조 [별표 2]와 관련하여, 편제 조정으로 인하여 이 학칙 시행당시 소속대학이 변경되거나 명칭이 변경된 학과(전공) 또는 신입생 모집이 중단된 학과(전공)의 재적생에 대하여는 구 학칙에 따라 다음과 같이 학사학위를 수여한다.  \n[\n    {'대학': '사회과학대학', '학과 또는 전공': '사회복지학', '학위의 종류': '<문학사>'},\n    {'대학': '자연과학대학', '학과 또는 전공': '분자생명과학', '학위의 종류': '<이학사>'},\n    {'대학': '공과대학', '학과 또는 전공': '컴퓨터학', '학위의 종류': '<공학사>'},\n    {'대학': '공과대학', '학과 또는 전공': '정보통신학', '학위의 종류': '<공학사>'},\n    {'대학': '공과대학', '학과 또는 전공': '건축학(4년제)', '학위의 종류': '<공학사>'},\n    {'대학': '공과대학', '학과 또는 전공': '건축학(5년제)', '학위의 종류': '<건축학사>'},\n    {'대학': '공과대학', '학과 또는 전공': '환경학', '학위의 종류': '<공학사>'},\n    {'대학': '음악대학', '학과 또는 전공': '건반악기', '학위의 종류': '<음악학사>'},\n    {'대학': '음악대학', '학과 또는 전공': '관현악', '학위의 종류': '음악학사'},\n    {'대학': '음악대학', '학과 또는 전공': '성악', '학위의 종류': '음악학사'},\n    {'대학': '음악대학', '학과 또는 전공': '교회음악', '학위의 종류': '음악학사'},\n    {'대학': '음악대학', '학과 또는 전공': '작곡', '학위

In [75]:
prompt_template = PromptTemplate.from_template(
    """
    Please provide the most correct answer ONLY BASED on the context.
    IMPORTANT: Select the EXACT, SAME answer from the question and the context!!!
    
    Response format(Must be strictly followed!):
    <Start>
    [Answer]: (A) answer
    [Reason]: Your short reason why
    <End>
    
    Now, here are the question and context:
    Question: {question}
    Context: {context}
    
    If the answer is NOT found, you MUST respond with "The information is not present in the context."
    """
)

In [76]:
chain = prompt_template | llm

context = get_article_content_text(title_idx, top3_article_idx, '../docs/doc_2.json')

print(context)
response = chain.invoke({"question": query, "context": context})

["부칙(2006. 6. 28  개정)\n ① (시행일)  이 학칙은 2007년 3월 1일부터 시행한다. 다만, 제8조 제4항 제18호는 2005년 8월 13일부터, 제50조 [별표 2] 중 바이오인포매틱스 연계전공은 2006년 9월 1일부터 시행한다. \n② (경과조치)  제50조 [별표 2]와 관련하여, 편제 조정으로 인하여 이 학칙 시행당시 소속대학이 변경되거나 명칭이 변경된 학과(전공) 또는 신입생 모집이 중단된 학과(전공)의 재적생에 대하여는 구 학칙에 따라 다음과 같이 학사학위를 수여한다.  \n[\n    {'대학': '사회과학대학', '학과 또는 전공': '사회복지학', '학위의 종류': '<문학사>'},\n    {'대학': '자연과학대학', '학과 또는 전공': '분자생명과학', '학위의 종류': '<이학사>'},\n    {'대학': '공과대학', '학과 또는 전공': '컴퓨터학', '학위의 종류': '<공학사>'},\n    {'대학': '공과대학', '학과 또는 전공': '정보통신학', '학위의 종류': '<공학사>'},\n    {'대학': '공과대학', '학과 또는 전공': '건축학(4년제)', '학위의 종류': '<공학사>'},\n    {'대학': '공과대학', '학과 또는 전공': '건축학(5년제)', '학위의 종류': '<건축학사>'},\n    {'대학': '공과대학', '학과 또는 전공': '환경학', '학위의 종류': '<공학사>'},\n    {'대학': '음악대학', '학과 또는 전공': '건반악기', '학위의 종류': '<음악학사>'},\n    {'대학': '음악대학', '학과 또는 전공': '관현악', '학위의 종류': '음악학사'},\n    {'대학': '음악대학', '학과 또는 전공': '성악', '학위의 종류': '음악학사'},\n    {'대학': '음악대학', '학과 또는 전공': '교회음악', '학위의 종류': '음악학사'},\n    {'대학': '음악대학', '학과 또는 전공': '작곡', '학위

In [77]:
print(response.content)

<Start>
[Answer]: (D) 체육학사
[Reason]: The context states that the degree type for a student in the Department of Social Physical Education is a Bachelor of Physical Education (<이학사>).
<End>
